In [ ]:
import tensorflow
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Conv2D, GlobalAveragePooling2D
from keras.models import Sequential, Model
from imgaug import parameters as iap
import imgaug as ia
import imgaug.augmenters as iaa
from keras.applications.mobilenet_v2 import preprocess_input
from keras.applications import DenseNet121, DenseNet169, MobileNet, MobileNetV2
from keras.optimizers import Adam
import pandas as pd
from timeit import default_timer as timer
from tqdm import tqdm
from keras.callbacks import Callback
from keras import applications
import inspect
from keras.models import Model
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from imgaug.augmenters import size
from imgaug.augmenters.arithmetic import Invert
from keras.utils import img_to_array
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

TRAIN_DIR = './mask_dataset/data/train/'
TEST_DIR = './mask_dataset/data/test/'

IM_HEIGHT, IM_WIDTH = 224, 224

def sometimes(aug): return iaa.Sometimes(0.1, aug)

seq_train = iaa.Sequential(
    [
        iaa.Fliplr(0.5),
        iaa.Flipud(0.2),
        sometimes(iaa.Crop(percent=(0, 0.1))),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
            rotate=(-45, 45),
            shear=(-16, 16),
            order=[0, 1],
            cval=(0, 255),
            mode=ia.ALL
        )),
        iaa.SomeOf((0, 5),
                   [
            sometimes(
                iaa.Superpixels(
                    p_replace=(0, 1.0),
                    n_segments=(20, 200)
                )
            ),
            iaa.OneOf([
                iaa.GaussianBlur((0, 3.0)),
                iaa.AverageBlur(k=(2, 7)),
                iaa.MedianBlur(k=(3, 11)),
            ]),
            iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),
            iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),
            sometimes(iaa.OneOf([
                iaa.EdgeDetect(alpha=(0, 0.7)),
                iaa.DirectedEdgeDetect(
                    alpha=(0, 0.7), direction=(0.0, 1.0)
                ),
            ])),
            iaa.AdditiveGaussianNoise(
                loc=0, scale=(0.0, 0.05*255), per_channel=0.5
            ),
            iaa.OneOf([
                iaa.Dropout((0.01, 0.1), per_channel=0.5),
                iaa.CoarseDropout(
                    (0.03, 0.15), size_percent=(0.02, 0.05),
                    per_channel=0.2
                ),
            ]),
            iaa.Invert(0.05, per_channel=True),
            iaa.Add((-10, 10), per_channel=0.5),
            iaa.Multiply((0.5, 1.5), per_channel=0.5),
            iaa.LinearContrast((0.5, 2.0), per_channel=0.5),
            iaa.Grayscale(alpha=(0.0, 1.0)),
            sometimes(
                iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)
            ),
            sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05)))
        ],
            random_order=True
        )
    ],
    random_order=True
)

def img_preprocessing_train(img):
    img = img.reshape(1, img.shape[0], img.shape[1], 3)
    img = img.astype(np.uint8)
    generate_img = seq_train(images=img)
    generate_img = generate_img/255.
    return generate_img.reshape(IM_HEIGHT, IM_WIDTH, 3).astype(np.float32)

train_datagen = ImageDataGenerator(
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.5, 1.2],
    zoom_range=[0.8, 1.2],
    preprocessing_function=img_preprocessing_train
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IM_HEIGHT, IM_WIDTH),
    batch_size=64,
    class_mode='categorical',
    shuffle=True
)

test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IM_HEIGHT, IM_WIDTH),
    batch_size=64,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Conv2D,GlobalAveragePooling2D
from keras.optimizers import Adam
from keras.applications import DenseNet121, DenseNet169, MobileNet, MobileNetV2
from keras.applications.mobilenet_v2 import preprocess_input


from keras.callbacks import ModelCheckpoint,EarlyStopping

cb_checkpoint = ModelCheckpoint(filepath="./DenseNet/", # 학습 진행시 가장 정확도가 높은 모델을 저장할 경로
                                monitor='val_acc', # 저장할 조건 val_acc (테스트 데이터의 정확도) 가 가장 높은 모델을 저장
                                vervose=1, #함수의 진행 과정 출력
                                save_best_only=True # 가장 정확도가 높은 모델 1개만 저장
                                )

base_Model = DenseNet121(include_top=False, input_shape = (IM_WIDTH, IM_HEIGHT,3), weights = 'imagenet')

x = base_Model.output

x = GlobalAveragePooling2D()(x)

x = Dense(128, activation="relu")(x)

x = Dropout(0.5)(x)

x = Dense(4, activation="softmax")(x)

model = Model(inputs=base_Model.input, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer=Adam(1e-05),metrics=['acc']) 

early_stop = EarlyStopping(monitor='val_loss', patience=10)

hist = model.fit(
      train_generator,
      epochs=100,
      validation_data=test_generator, # 한번 학습이 끝날때 마다 test_generator(테스트 데이터) 로 검증
      callbacks=[cb_checkpoint,early_stop] # 학습시 가장 정확도가 높은 모델을 저장하도록 설정, 10번 안에 val_loss가 개선되지않으면 중지
      )

# 모델 학습 과정 표시하기
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(hist.history['loss'], 'y', label='train loss')
loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')

acc_ax.plot(hist.history['acc'], 'b', label='train acc')
acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

In [ ]:
import keras
#저장한 모델을 읽어서 best_model 변수에 대입
best_model = keras.models.load_model('./DenseNet/')
#best_model을 이용해 정확도 계산
best_model.evaluate(test_generator)